In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install -q datasets evaluate rouge_score accelerate --quiet
!pip install transformers --quiet
!pip install rouge_score --quiet
!pip install jiwer --quiet
!pip install git+https://github.com/google-research/bleurt.git --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 98.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/12

In [3]:
import transformers
from huggingface_hub import notebook_login
from evaluate import load
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk
import numpy as np
import pandas as pd
import accelerate
import torch
from torch.utils.data import Dataset



In [7]:
notebook_login()

In [5]:
# ====== ENVIRONMENT ======
DEV = False
EXPLORE = False
RANDOM_SEED = 42

# ====== DATA =======
INPUT_MAX_LENGTH = 512
LABEL_MAX_LENGTH = 128

# ====== MODEL ======
CHECKPOINT = "t5-base"
MODEL_NAME = "t-5-base-baseline"
HUGGINGFACE_DIR = "jgibb/" + MODEL_NAME
PATH = '/content/drive/MyDrive/colab-notebooks/w266_project/'
PEFT = False

# ====== OPTIIZER =======
OPTIMIZER = "adamw_bnb_8bit"
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 0.01

# ====== TRAINING ======
EVAL_STRATEGY = "steps"
BATCH_SIZE = 6
EPOCHS = 2
SAVE_STEPS = 100 if DEV else 1000
EVAL_STEPS = 100 if DEV else 250
LOGGING_STEPS = 100 if DEV else 500
METRIC_WER = load("wer")
METRIC_ROUGE = load("rouge")
METRIC_BLEURT = load('bleurt', model_type = 'metric', checkpoint = "bleurt-base-512")

In [6]:
dataset = load_dataset("jordanfan/processed_us_congress_117_bills_v3")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11277 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/3388 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/377 [00:00<?, ? examples/s]

In [8]:
tokenizer = T5Tokenizer.from_pretrained(CHECKPOINT)
model = T5ForConditionalGeneration.from_pretrained(CHECKPOINT)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
def preprocess_data_baseline(df):
  input = ["summarize: " + doc for doc in df["cur_text"]]
  #tokenize full text
  model_inputs = tokenizer(input, max_length=INPUT_MAX_LENGTH, return_tensors = "pt", padding=True, truncation=True)
  #tokenize the summaries
  summary_input = [doc for doc in df["cleaned_summary"]]
  targets = tokenizer(text_target = summary_input, max_length=LABEL_MAX_LENGTH, padding= True, truncation=True)
  #set labels
  model_inputs['labels'] = targets['input_ids']
  return model_inputs

def preprocess_data_extracted(df):
  input = ["summarize: " + doc for doc in df["billsum_abstracted_500"]]
  #tokenize full text
  model_inputs = tokenizer(input, max_length=INPUT_MAX_LENGTH, padding=True, truncation=True)
  #tokenize the summaries
  summary_input = [doc for doc in df["cleaned_summary"]]
  targets = tokenizer(text_target = summary_input, max_length=LABEL_MAX_LENGTH, padding= True, truncation=True)
  #set labels
  model_inputs['labels'] = targets['input_ids']
  return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_data_baseline, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=CHECKPOINT)

Map:   0%|          | 0/11277 [00:00<?, ? examples/s]

Map:   0%|          | 0/3388 [00:00<?, ? examples/s]

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions[0]

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    result_wer = METRIC_WER.compute(predictions=pred_str, references=label_str)
    result_rouge = METRIC_ROUGE.compute(predictions=pred_str, references=label_str, use_stemmer=True, use_aggregator=True)
    result_rouge = {key: value for key, value in result_rouge.items()}
    result_bleurt = METRIC_BLEURT.compute(predictions = pred_str, references = label_str)

    results = {key: value for key, value in result_rouge.items()}
    results["WER"] = result_wer
    results["BLEURT"] = result_bleurt["scores"][0]
    return {k: round(v, 4) for k,v in results.items()}

def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=HUGGINGFACE_DIR,
    evaluation_strategy=EVAL_STRATEGY,
    save_strategy=EVAL_STRATEGY,
    save_steps=SAVE_STEPS,
    eval_steps=EVAL_STEPS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=WEIGHT_DECAY,
    load_best_model_at_end=True,
    num_train_epochs=EPOCHS,
    # predict_with_generate=True,
    fp16=True,
    report_to=["tensorboard"],
    push_to_hub=True,
)

In [ ]:

def model_init():
  return T5ForConditionalGeneration.from_pretrained(CHECKPOINT)

trainer = Seq2SeqTrainer(
    #model=model,
    model_init= model_init,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics = preprocess_logits_for_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Wer,Bleurt
250,No log,1.331624,0.651100,0.376900,0.586800,0.586600,0.521700,0.300900
500,1.791900,1.277646,0.659500,0.386600,0.596400,0.596200,0.510800,0.300900
750,1.791900,1.251330,0.663500,0.393200,0.601600,0.601400,0.503900,0.300900
1000,1.355200,1.232581,0.666800,0.396800,0.605000,0.604800,0.500800,0.300900
1250,1.355200,1.223553,0.669200,0.400000,0.607300,0.607200,0.497200,0.331400
1500,1.307400,1.211822,0.671300,0.402300,0.609400,0.609300,0.495300,0.331400
1750,1.307400,1.202176,0.671600,0.403500,0.610600,0.610500,0.493200,0.279800
2000,1.303700,1.197202,0.673100,0.405300,0.611800,0.611700,0.491600,0.377100
2250,1.303700,1.190858,0.675000,0.406900,0.613600,0.613500,0.490500,0.331400
2500,1.267600,1.188912,0.676100,0.408700,0.614400,0.614300,0.489300,0.331400


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=3760, training_loss=1.3563128856902427, metrics={'train_runtime': 7654.9885, 'train_samples_per_second': 2.946, 'train_steps_per_second': 0.491, 'total_flos': 1.373443626369024e+16, 'train_loss': 1.3563128856902427, 'epoch': 2.0})

In [ ]:
predict_results = trainer.predict(tokenized_datasets["test"])
trainer.save_metrics("test", predict_results.metrics)

In [ ]:
commit_msg = f"training completed[dev]: {INPUT_MAX_LENGTH} {LABEL_MAX_LENGTH} "
trainer.push_to_hub(commit_message=commit_msg)

events.out.tfevents.1712366264.b7d2b20d960d.1095.0:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jgibb/t-5-base-baseline/commit/004ba6aad57e36c3dda53f02f1a23f9f919f75de', commit_message='training completed[dev]: 512 128 ', commit_description='', oid='004ba6aad57e36c3dda53f02f1a23f9f919f75de', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
print(predict_results.metrics)

In [ ]:
import pprint
pprint.pprint(predict_results.metrics)